In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler

In [3]:
import matplotlib.pyplot as plt 
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings('ignore')

In [4]:
import pickle

In [5]:
Cough_dataframe = pd.read_csv("./Cough.csv")
Wheeze_dataframe = pd.read_csv("./Wheeze.csv")
Aaa_dataframe = pd.read_csv("./Aaa.csv")
Yee_dataframe = pd.read_csv("./Yee.csv")
Inhale_dataframe = pd.read_csv("./Inhale.csv")
Exhale_dataframe = pd.read_csv("./Exhale.csv")
Ooo_dataframe = pd.read_csv("./Ooo.csv")
Sss_dataframe = pd.read_csv("./Sss.csv")
Eee_dataframe = pd.read_csv("./Eee.csv")
Uuu_dataframe = pd.read_csv("./Uuu.csv")
Zzz_dataframe = pd.read_csv("./Zzz.csv")

FileNotFoundError: [Errno 2] No such file or directory: './Wheeze.csv'

In [9]:
sets_loc = './SETS/'

In [10]:
def fold(sound_dataframe, sets_loc):
    big = []
    for j in os.listdir(sets_loc):      

        file = open(sets_loc+j, 'r') 
        Lines = file.readlines() 

        count = 0
        # Strips the newline character 
        # for line in Lines: 
        #     print("Line{}: {}".format(count, line.strip())) 

        set_dataframe = pd.DataFrame()
        for i, line in enumerate(Lines):
            print(line)
            A = sound_dataframe[(sound_dataframe['73'] == (Lines[i].strip()+".json"))]
            set_dataframe = pd.DataFrame.append(set_dataframe,A)
        
        big.append(set_dataframe)
    fold_1 = shuffle(pd.concat(big[:4]))
    test_1 = shuffle(big[4])
    fold_2 = shuffle(pd.concat(big[1:5]))
    test_2 = shuffle(big[0])
    fold_3 = shuffle(pd.concat([big[0],big[2],big[3],big[4]]))
    test_3 = shuffle(big[1])
    fold_4 = shuffle(pd.concat([big[0],big[1],big[3],big[4]]))
    test_4 = shuffle(big[2])
    fold_5 = shuffle(pd.concat([big[0],big[2],big[1],big[4]]))
    test_5 = shuffle(big[3])
    
    return fold_1,test_1,fold_2,test_2,fold_3,test_3,fold_4,test_4,fold_5,test_5

In [11]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5 = fold(Cough_dataframe,sets_loc)

2192_P_Asthma_M_before_Mil_Y

2244_P_Asthma_F_before_Mod_Y

2297_P_Asthma_M_before_Mod_Y

2394_P_Asthma_M_before_Mod_Y

2224_P_Asthma_M_before_Mil_Y

2216_P_Asthma_F_before_Mod_Y

2219_P_Asthma_F_before_Mod_Y

2469_P_Asthma_M_before_Mil_Y

2212_P_Asthma_F_before_Mod_Y

1452_C_Na_M_Na_Na_Y

1174_C_Na_M_Na_Na_Y

1055_C_Na_F_Na_Na_N

1399_C_Na_F_Na_Na_N

1101_C_Na_F_Na_Na_Y

1186_C_Na_M_Na_Na_Y

1280_C_Na_M_Na_Na_Y

1211_C_Na_M_Na_Na_Y

1074_C_Na_F_Na_Na_Y

2153_P_Asthma_M_before_Sev_Y

2258_P_Asthma_M_before_Sev_Y

2178_P_Asthma_M_before_Mod_Y

2172_P_Asthma_M_before_Sev_Y

2324_P_Asthma_M_before_Mod_Y

2135_P_Asthma_F_before_Sev_Y

2398_P_Asthma_F_before_Sev_Y

2238_P_Asthma_F_before_Mod_Y

2100_P_Asthma_M_before_Mil_Y

2465_P_Asthma_M_before_Sev_Y

1123_C_Na_M_Na_Na_Y

1035_C_Na_M_Na_Na_N

1265_C_Na_F_Na_Na_N

1320_C_Na_F_Na_Na_N

1400_C_Na_M_Na_Na_Y

1184_C_Na_M_Na_Na_Y

1338_C_Na_F_Na_Na_Y

1163_C_Na_F_Na_Na_Y

1381_C_Na_F_Na_Na_Y

1499_C_Na_M_Na_Na_Y

2182_P_Asthma_F_before_Mod_Y

2

In [8]:
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5]

In [10]:
def SVM(fold_list):
    list_train = []
    list_test = []
    for i in range(0,len(fold_list),2):
        fold = fold_list[i]
        test = fold_list[i+1]
        X_train = fold.iloc[:,:72]
        y_train = fold.iloc[:,72]
        X_test = test.iloc[:,:72]
        y_test = test.iloc[:,72]


        scaler = StandardScaler()

        X_train = scaler.fit_transform(X_train)

        X_test = scaler.transform(X_test)

        X_train = pd.DataFrame(X_train)
        X_test = pd.DataFrame(X_test)



        ### SVM

        svm = SVC()

        svm.fit(X_train,y_train)
        
#         pickle.dump(svm, open('./Wheeze.model', 'wb'))


        list_test.append(accuracy_score(y_test,svm.predict(X_test)))

        list_train.append(accuracy_score(y_train,svm.predict(X_train)))
        
    return list_test, list_train

In [12]:
cough_test, cough_train = SVM(fold_list)

In [15]:
from sklearn.model_selection import StratifiedShuffleSplit

In [9]:
def hyperparameter_tuning(fold,test):
    X_train = fold.iloc[:,:78]
    y_train = fold.iloc[:,78]    
    C_range = np.logspace(-1, 10, 12)
    gamma_range = np.logspace(-1, 20, 22)
    param_grid = dict(gamma=gamma_range, C=C_range)
    cv = StratifiedShuffleSplit(n_splits=5)
    grid = GridSearchCV(SVC(), param_grid=param_grid, cv=cv, verbose=3, n_jobs=-1)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    grid.fit(X_train, y_train)
    print(grid.best_params_)
    X_test = test.iloc[:,:78]
    X_test = scaler.transform(X_test)
    y_pred = grid.predict(X_test)
    a = accuracy_score(test.iloc[:,78],y_pred)
    return a


In [53]:
tune_acc_cough = []
for i in range(0,len(fold_list),2):
    tune_acc_cough.append(hyperparameter_tuning(fold_list[i],fold_list[i+1]))

Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 278 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 918 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   10.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


...

[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    4.9s


{'C': 10.0, 'gamma': 0.1}


[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.3s finished


In [57]:
np.mean(cough_test)

0.6676646182353787

In [20]:
cough_test

[0.6612903225806451,
 0.7209302325581395,
 0.589622641509434,
 0.668918918918919,
 0.697560975609756]

In [21]:
np.std(cough_test)

0.044415701953316386

In [58]:
(tune_acc_cough)

[0.6896551724137931,
 0.6046511627906976,
 0.49523809523809526,
 0.5172413793103449,
 0.47474747474747475]

In [59]:
np.mean(tune_acc_cough)

0.5563066569000812

In [60]:
a = [1,2,3,4]

In [61]:
np.std(tune_acc_cough)

0.08003297216716057

### Wheeze

In [11]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5 = fold(Wheeze_dataframe,sets_loc)
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5]
# wheeze_test, wheeze_train = SVM(fold_list)


In [10]:
f1.to_csv('f1.csv', index=False, header=True)
# t1.to_csv(index=False, header=False)

In [11]:
f1 = pd.read_csv('./f1.csv')

In [12]:
f1

,0,1,2,3,4,5,6,7,8,9,...,64,65,66,67,68,69,70,71,72,73
0,2.348842,-11.050487,-5.923552,-0.962313,-0.753902,-0.913334,-6.036192,-17.630626,0.708860,3.504011,...,11.275560,11.175546,12.270526,21.610388,10.128147,11.310135,9.861360,10.838769,0.0,1101_C_Na_F_Na_Na_Y.json
1,2.935443,-18.499336,-13.845093,-11.418120,-5.498770,-17.342289,-5.322224,-8.994167,23.229160,9.451137,...,11.486836,21.299410,10.039448,11.905370,26.964476,18.734164,13.045341,12.044536,1.0,2216_P_Asthma_F_before_Mod_Y.json
2,-0.827957,-10.444054,-8.788164,-10.281637,3.832704,-5.810532,-3.211560,-2.993491,-14.661019,23.812566,...,10.099070,12.404433,10.154532,12.736116,17.461141,27.395012,16.619812,14.697697,0.0,1400_C_Na_M_Na_Na_Y.json
3,2.988880,2.555157,-11.234648,-18.552736,0.192577,-10.045409,-18.164909,-16.248890,-34.731943,12.952533,...,8.758464,14.402351,20.761831,21.805710,38.925252,17.484403,8.599152,14.463948,1.0,2172_P_Asthma_M_before_Sev_Y.json
4,-1.550135,-11.235290,-13.002058,-16.906151,0.926294,-26.884304,-21.351739,-17.471478,-11.799591,11.549982,...,11.078813,28.976152,22.729314,26.237326,16.659200,17.493651,15.569850,12.244570,0.0,1469_C_Na_M_Na_Na_Y.json
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732,1.768303,-14.930396,-3.763726,-14.677401,2.876948,-9.873451,-0.738448,-11.603225,11.795650,12.428571,...,9.509380,12.553271,11.270999,21.764950,19.895629,17.839932,18.292663,19.316942,1.0,2273_P_Asthma_F_before_Mil_Y.json
733,-3.144785,-8.558714,-18.062176,-7.570649,-7.149041,-5.977456,-3.704684,-13.303762,-1.852671,1.678716,...,10.714963,13.706303,18.472630,21.919663,14.673877,11.594331,15.392582,15.215450,0.0,1265_C_Na_F_Na_Na_N.json
734,-0.887440,-5.171603,-6.323254,0.537335,-1.121832,-11.409050,-7.459699,-6.955061,-5.467595,-0.419238,...,6.932143,13.912016,11.176017,13.249154,11.281815,18.722704,8.427402,11.649862,0.0,1176_C_Na_M_Na_Na_Y.json
735,-1.407821,-6.693597,-12.129014,-3.806145,-15.863923,5.227487,-6.173265,-13.902049,-5.944600,0.196814,...,22.749432,18.153218,16.485457,23.099977,19.387218,13.603509,14.871958,13.672343,0.0,1265_C_Na_F_Na_Na_N.json


In [14]:
X_train = fold.iloc[:,:72]
y_train = fold.iloc[:,72]
X_test = test.iloc[:,:72]
y_test = test.iloc[:,72]


scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)



### SVM

svm = SVC()

svm.fit(X_train,y_train)

pickle.dump(svm, open('./Wheeze.model', 'wb'))


In [15]:
wheeze_test, wheeze_train

([0.8881578947368421,
  0.7575757575757576,
  0.7153284671532847,
  0.6608695652173913,
  0.608],
 [0.9783889980353635,
  0.9773156899810964,
  0.982824427480916,
  0.9706959706959707,
  0.9813432835820896])

### saving model

In [12]:
import pickle
# filename = 'finalized_model.sav'
# pickle.dump(model, open(filename, 'wb'))

In [64]:
np.mean(wheeze_test)

0.7229817804226095

In [65]:
np.std(wheeze_test)

0.09852819478817772

In [66]:
tune_acc_wheeze = []
for i in range(0,len(fold_list),2):
    tune_acc_wheeze.append(hyperparameter_tuning(fold_list[i],fold_list[i+1]))

Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 400 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 1040 tasks      | elapsed:   13.0s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   16.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 400 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done 1040 tasks      | elapsed:   12.8s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   16.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


...

[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 208 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done 528 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 976 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   17.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 400 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 1040 tasks      | elapsed:   13.0s


{'C': 1.0, 'gamma': 0.1}


[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   16.5s finished


In [67]:
tune_acc_wheeze

[0.42105263157894735,
 0.6363636363636364,
 0.5693430656934306,
 0.5652173913043478,
 0.528]

In [68]:
np.mean(tune_acc_wheeze)

0.5439953449880723

In [69]:
np.std(tune_acc_wheeze)

0.07070437792404904

### Aaa 

In [23]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5 = fold(Aaa_dataframe,sets_loc)
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5]
Aaa_test, Aaa_train = SVM(fold_list)


In [27]:
(Aaa_test)

([0.5257731958762887,
  0.6024096385542169,
  0.6320754716981132,
  0.42528735632183906,
  0.7128712871287128],)

In [28]:
np.std(Aaa_test)

0.09771885466539448

In [72]:
tune_acc_Aaa = []
for i in range(0,len(fold_list),2):
    tune_acc_Aaa.append(hyperparameter_tuning(fold_list[i],fold_list[i+1]))

Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 1.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    9.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


...

[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    9.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 1.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    3.8s


{'C': 10.0, 'gamma': 0.1}


[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.6s finished


In [73]:
np.mean(tune_acc_Aaa)

0.5347037183808909

In [74]:
tune_acc_Aaa

[0.5463917525773195,
 0.5662650602409639,
 0.5188679245283019,
 0.5172413793103449,
 0.5247524752475248]

In [75]:
np.std(tune_acc_Aaa)

0.01890746591186171

### Eee

In [76]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5 = fold(Eee_dataframe,sets_loc)
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5]
Eee_test, Eee_train = SVM(fold_list)


In [77]:
Eee_test, Eee_train

([0.7586206896551724,
  0.6046511627906976,
  0.6571428571428571,
  0.6551724137931034,
  0.6868686868686869],
 [0.9628647214854111,
  0.9629629629629629,
  0.9721448467966574,
  0.9681697612732095,
  0.9616438356164384])

In [78]:
np.mean(Eee_test)

0.6724911620501034

In [79]:
tune_acc_Eee = []
for i in range(0,len(fold_list),2):
    tune_acc_Eee.append(hyperparameter_tuning(fold_list[i],fold_list[i+1]))

Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 400 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 1040 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s


{'C': 1.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


...

[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 1.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    3.7s


{'C': 10.0, 'gamma': 0.1}


[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.3s finished


In [80]:
tune_acc_Eee

[0.6896551724137931,
 0.6046511627906976,
 0.49523809523809526,
 0.5172413793103449,
 0.47474747474747475]

In [81]:
np.mean(tune_acc_Eee)

0.5563066569000812

In [82]:
np.std(Eee_test)

0.05052283394587567

### Yee

In [83]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5 = fold(Yee_dataframe,sets_loc)
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5]
Yee_test, Yee_train = SVM(fold_list)

In [84]:
Yee_test, Yee_train

([0.8658536585365854,
  0.6091954022988506,
  0.7019230769230769,
  0.6511627906976745,
  0.6263736263736264],
 [0.9456521739130435,
  0.9724517906336089,
  0.9797687861271677,
  0.9697802197802198,
  0.9693593314763231])

In [85]:
np.mean(Yee_test)

0.6909017109659629

In [86]:
tune_acc_Yee = []
for i in range(0,len(fold_list),2):
    tune_acc_Yee.append(hyperparameter_tuning(fold_list[i],fold_list[i+1]))

Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


...

[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 1.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    4.7s


{'C': 10.0, 'gamma': 0.1}


[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.0s finished


In [87]:
tune_acc_Yee

[0.5609756097560976,
 0.4942528735632184,
 0.5192307692307693,
 0.5465116279069767,
 0.4175824175824176]

In [88]:
np.mean(tune_acc_Yee)

0.507710659607896

In [89]:
np.mean(Yee_test)


0.6909017109659629

In [90]:
np.std(Yee_test)

0.09289631944007794

### Inhale

In [18]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5 = fold(Inhale_dataframe,sets_loc)
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5]
Inhale_test, Inhale_train = SVM(fold_list)


In [72]:
tune_acc_Inhale = []
for i in range(0,len(fold_list),2):
    tune_acc_Inhale.append(hyperparameter_tuning(fold_list[i],fold_list[i+1]))

Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:   12.3s
[Parallel(n_jobs=-1)]: Done 496 tasks      | elapsed:   16.8s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   22.6s
[Parallel(n_jobs=-1)]: Done 1136 tasks      | elapsed:   29.7s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   33.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 208 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 528 tasks      | elapsed:   11.5s
[Parallel(n_jobs=-1)]: Done 976 tasks      | elapsed:   21.4s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   29.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


...

[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 208 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 528 tasks      | elapsed:   11.3s
[Parallel(n_jobs=-1)]: Done 976 tasks      | elapsed:   20.9s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   28.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 208 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 528 tasks      | elapsed:   10.9s
[Parallel(n_jobs=-1)]: Done 976 tasks      | elapsed:   20.5s


{'C': 10.0, 'gamma': 0.1}


[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   27.6s finished


In [73]:
np.mean(tune_acc_Inhale)

0.6271087440735001

In [19]:
np.mean(Inhale_test)

0.7650516491527778

In [20]:
Inhale_test

[0.8121546961325967,
 0.7848101265822784,
 0.7473118279569892,
 0.6809815950920245,
 0.8]

In [76]:
tune_acc_Inhale

[0.6132596685082873,
 0.5379746835443038,
 0.6182795698924731,
 0.754601226993865,
 0.6114285714285714]

In [95]:
np.std(tune_acc_Inhale)

0.07030623439287106

### Exhale

In [33]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5 = fold(Exhale_dataframe,sets_loc)
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5]
Exhale_test, Exhale_train = SVM(fold_list)


In [ ]:
tune_acc_Exhale = []
for i in range(0,len(fold_list),2):
    tune_acc_Exhale.append(hyperparameter_tuning(fold_list[i],fold_list[i+1]))

In [34]:
np.mean(Exhale_test)

0.7018601215206426

In [35]:
Exhale_test

[0.7734806629834254,
 0.7388535031847133,
 0.6935483870967742,
 0.6748466257668712,
 0.6285714285714286]

In [36]:
np.std(Exhale_test)

0.05033499506812715

In [98]:
tune_acc_Exhale

[0.7458563535911602,
 0.7261146496815286,
 0.5913978494623656,
 0.6932515337423313,
 0.5771428571428572]

In [99]:
np.mean(tune_acc_Exhale)

0.6667526487240485

In [100]:
np.std(tune_acc_Exhale)

0.0695581694807795

### Ooo

In [101]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5 = fold(Ooo_dataframe,sets_loc)
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5]
Ooo_test, Ooo_train = SVM(fold_list)
tune_acc_Ooo = []
for i in range(0,len(fold_list),2):
    tune_acc_Ooo.append(hyperparameter_tuning(fold_list[i],fold_list[i+1]))

Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   10.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   10.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


...

[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    4.2s


{'C': 10.0, 'gamma': 0.1}


[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.9s finished


In [102]:
np.mean(Ooo_test)

0.6541669089288137

In [103]:
Ooo_test

[0.6419753086419753,
 0.6904761904761905,
 0.6730769230769231,
 0.5918367346938775,
 0.673469387755102]

In [104]:
np.std(Ooo_test)

0.034877568688540264

In [105]:
np.mean(tune_acc_Ooo)

0.5452579607341512

### Sss

In [37]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5 = fold(Sss_dataframe,sets_loc)
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5]
Sss_test, Sss_train = SVM(fold_list)
tune_acc_Sss = []
for i in range(0,len(fold_list),2):
    tune_acc_Sss.append(hyperparameter_tuning(fold_list[i],fold_list[i+1]))

Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   26.2s
[Parallel(n_jobs=-1)]: Done 328 tasks      | elapsed:   27.9s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   33.2s finished


{'C': 1.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    7.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


...

[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   13.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    4.3s


{'C': 10.0, 'gamma': 0.1}


[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    7.2s finished


In [38]:
np.mean(Sss_test)

0.7261143116266368

In [40]:
Sss_test

[0.717948717948718,
 0.7662337662337663,
 0.73,
 0.631578947368421,
 0.7848101265822784]

In [41]:
tune_acc_Sss

[0.5512820512820513,
 0.7012987012987013,
 0.59,
 0.631578947368421,
 0.4810126582278481]

In [42]:
np.mean(tune_acc_Sss)

0.5910344716354043

In [43]:
np.std(tune_acc_Sss)

0.07414604417864344

In [44]:
np.std(Sss_test)

0.05305370848670571

### Uuu

In [111]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5 = fold(Uuu_dataframe,sets_loc)
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5]
Uuu_test, Uuu_train = SVM(fold_list)
tune_acc_Uuu = []
for i in range(0,len(fold_list),2):
    tune_acc_Uuu.append(hyperparameter_tuning(fold_list[i],fold_list[i+1]))

Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


...

[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    4.6s


{'C': 10.0, 'gamma': 0.1}


[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    7.7s finished


In [112]:
Uuu_test

[0.5882352941176471,
 0.49411764705882355,
 0.5161290322580645,
 0.620253164556962,
 0.6534653465346535]

In [113]:
np.mean(Uuu_test)

0.5744400969052301

In [114]:
np.std(Uuu_test)

0.06063993885616763

In [115]:
np.mean(tune_acc_Uuu)

0.5267919211396076

### Zzz

In [15]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5 = fold(Zzz_dataframe,sets_loc)
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5]
Zzz_test, Zzz_train = SVM(fold_list)
tune_acc_Zzz = []
for i in range(0,len(fold_list),2):
    tune_acc_Zzz.append(hyperparameter_tuning(fold_list[i],fold_list[i+1]))

Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 383 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    7.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done 976 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    6.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


...

[Parallel(n_jobs=-1)]: Done 976 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    5.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 1.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done 976 tasks      | elapsed:    4.5s


{'C': 10.0, 'gamma': 0.1}


[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    6.0s finished


In [16]:
np.mean(Zzz_test)

0.5425824832276446

In [17]:
tune_acc_Zzz

[0.6172839506172839,
 0.6,
 0.4731182795698925,
 0.5694444444444444,
 0.5230769230769231]

In [18]:
np.mean(tune_acc_Zzz)

0.5565847195417087

In [19]:
np.std(tune_acc_Zzz)

0.0525815077389504

LiBSVM

In [73]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5 = fold(Inhale_dataframe,sets_loc)
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5]

In [74]:
X_train = f1.iloc[:,:78]
y_train = f1.iloc[:,78]

In [67]:
X_train = (X_train-np.mean(X_train))/np.std(X_train)

In [38]:
from libsvm.svmutil import *

In [39]:
prob = svm_problem(y_train.values,X_train.values)

In [40]:
param = svm_parameter('-t 2 -c 1 -g 0.01282051282051282 -b 1')

In [41]:
m = libsvm.svm_train(prob, param)

In [42]:
svm_save_model('Inhale.model', m)

In [64]:
m = svm_load_model('Inhale.model')

In [69]:
p_label, p_acc, p_val = svm_predict(y_train.values, X_train.values, m, '-b 1')

Accuracy = 93.9883% (641/682) (classification)


In [76]:
p_label, p_acc, p_val = svm_predict(t1.iloc[:,78].values,t1.iloc[:,:78].values, m, '-b 1')

Accuracy = 81.2155% (147/181) (classification)


In [75]:
t1.iloc[:,:78] = (t1.iloc[:,:78]-np.mean(X_train))/np.std(X_train)

In [55]:
pd.read_csv("./mean_values")

,1.000000000000000444e+00
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
72,1.0
73,1.0
74,1.0
75,1.0


In [77]:
np.savetxt('./mean_values.txt', np.mean(X_train),fmt='%f' )

In [78]:
np.savetxt('./std_values.txt',np.std(X_train), fmt='%f')

In [8]:
C_range = np.logspace(-2, 3, 12)


In [9]:
C_range

array([1.00000000e-02, 2.84803587e-02, 8.11130831e-02, 2.31012970e-01,
       6.57933225e-01, 1.87381742e+00, 5.33669923e+00, 1.51991108e+01,
       4.32876128e+01, 1.23284674e+02, 3.51119173e+02, 1.00000000e+03])

In [10]:
12*32

384

In [ ]:
12